# USGS API
My own "tutorial".

The idea is to send url queries to the usgs db

In [1]:
from lxml import html
import requests

First I study the GeoJson format:

http://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php

In [2]:
# s = "http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_week.geojson"
s = "http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson"
# s = "http://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2014-01-01&endtime=2014-01-02"
r = requests.get(s)

Convert the json string to a dictionary

In [3]:
import json
d = json.loads(r.content)
d.keys()

[u'type', u'features', u'bbox', u'metadata']

As is shown in the website address mentioned above, each file is of this form:

```json
{
  type: "FeatureCollection",
  metadata: { 
  ...
  },
  bbox: [
    ...
  ],
  features: [
    {
      type: "Feature",
      properties: {
        mag: Decimal,
        ...
        tsunami: Integer,
        ...
        magType: String,
        type: String
      },
      geometry: {
        type: "Point",
        coordinates: [
          longitude,
          latitude,
          depth
        ]
      },
      id: String
    },
    …
  ]
}
```

In [4]:
for f in d['features']:
    coords = f['geometry']['coordinates']
    place = f['properties']['place']
    mag = f['properties']['mag']
    magType = f['properties']['magType'] 
    tsunamiFlag = f['properties']['tsunami']
    Type = f['properties']['type']
    print 'place = %s \n\tcoords ='%place,coords
    print '\tmag = %.3f%sm tsunami = %i Type = %s'%(mag,magType,tsunamiFlag, Type)

place = 109km E of Ile Hunter, New Caledonia 
	coords = [173.1114, -22.4953, 9.95]
	mag = 7.200mwwm tsunami = 1 Type = earthquake
place = 20km NNE of Upper Lake, California 
	coords = [-122.8018333, 39.3293333, 14.45]
	mag = 5.090mwm tsunami = 1 Type = earthquake
place = 70km ENE of Iwo Jima, Japan 
	coords = [142.0074, 24.9477, 510]
	mag = 6.300mwwm tsunami = 0 Type = earthquake
place = 31km ESE of Janesville, California 
	coords = [-120.2131667, 40.1538333, 6.87]
	mag = 4.480mwm tsunami = 0 Type = earthquake
place = 28km SSW of Agrihan, Northern Mariana Islands 
	coords = [145.541, 18.5439, 207.62]
	mag = 7.700mwwm tsunami = 1 Type = earthquake
place = 128km SE of Lorengau, Papua New Guinea 
	coords = [147.9672, -2.9523, 7.31]
	mag = 6.400mwwm tsunami = 1 Type = earthquake
place = 19km SE of Bayside, California 
	coords = [-123.8918333, 40.7243333, 26.24]
	mag = 4.730mwm tsunami = 1 Type = earthquake
place = 13km SSE of Ridgemark, California 
	coords = [-121.3296667, 36.6925, 5.87]
	

# Sending Queries to the USGS API

The API can give many kind of answers (methods), as is well explained here

http://earthquake.usgs.gov/fdsnws/event/1/

The first kind of interest is the "count" method which indicates how many results are associated to the querystring

http://earthquake.usgs.gov/fdsnws/event/1/count?format=geojson&starttime=2014-01-01&endtime=2014-01-02

The other is the "query" method, which actually performs the query

http://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2014-01-01&endtime=2014-01-02

Two kind of queries that can be interesting are:
1. Get all events bigger than a certain magnitude, considering the whole planet.
2. Given a point location, return all events "close" to that point, or the closest one. Close means a predefined radius.

For now, only the first kind of query.

## Query all events with magnitude over 9.0

In [5]:
baseCountString = "http://earthquake.usgs.gov/fdsnws/event/1/count?format=geojson"
baseQueryString = "http://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
startTime = "&starttime=0000-01-01"
endTime = "&endtime=2016-01-02"
minMagnitudeString = "&minmagnitude=8.0"
qString = baseQueryString + startTime + endTime + minMagnitudeString
cString = baseCountString + startTime + endTime + minMagnitudeString

In [6]:
countMag9 = requests.get(cString)
dCountMag9 = json.loads(countMag9.content)
dCountMag9

{u'count': 92, u'maxAllowed': 20000}

In [7]:
queryMag9 = requests.get(qString)
dQueryMag9 = json.loads(queryMag9.content)

In [8]:
for f in dQueryMag9['features']:
    coords = f['geometry']['coordinates']
    place = f['properties']['place']
    mag = f['properties']['mag']
    magType = f['properties']['magType'] 
    tsunamiFlag = f['properties']['tsunami']
    Type = f['properties']['type']
    print 'place = %s \n\tcoords ='%place,coords
    print '\tmag = %.3f%sm tsunami = %i Type = %s'%(mag,magType,tsunamiFlag, Type)

place = 48km W of Illapel, Chile 
	coords = [-71.6744, -31.5729, 22.44]
	mag = 8.300mwwm tsunami = 1 Type = earthquake
place = 94km NW of Iquique, Chile 
	coords = [-70.7691, -19.6097, 25]
	mag = 8.200mwwm tsunami = 1 Type = earthquake
place = Sea of Okhotsk 
	coords = [153.221, 54.892, 598.1]
	mag = 8.300mwwm tsunami = 1 Type = earthquake
place = 76km W of Lata, Solomon Islands 
	coords = [165.114, -10.799, 24]
	mag = 8.000mwwm tsunami = 1 Type = earthquake
place = off the west coast of northern Sumatra 
	coords = [92.463, 0.802, 25.1]
	mag = 8.200mwcm tsunami = 0 Type = earthquake
place = off the west coast of northern Sumatra 
	coords = [93.063, 2.327, 20]
	mag = 8.600mwwm tsunami = 0 Type = earthquake
place = near the east coast of Honshu, Japan 
	coords = [142.373, 38.297, 29]
	mag = 9.000mwwm tsunami = 0 Type = earthquake
place = offshore Bio-Bio, Chile 
	coords = [-72.898, -36.122, 22.9]
	mag = 8.800mwcm tsunami = 0 Type = earthquake
place = Samoa Islands region 
	coords = [-172